<a href="https://colab.research.google.com/github/krakowiakpawel9/decision_tree_course/blob/master/05_decision_tree_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

# https://medium.com/@rakendd/decision-tree-from-scratch-9e23bcfb4928

In [0]:
# najmniejsza reprezentowalna liczba
epsilon = np.finfo(float).eps

In [0]:
dataset = {'Dochód':['niski','średni','średni','średni','średni','wysoki','niski','wysoki','średni','niski'],
       'Wiek':[25, 36, 36, 25 ,28, 28, 25, 25, 36 ,36],
       'Wiarygodność':['niska','średnia','wysoka','niska','średnia','wysoka','niska','wysoka','średnia','średnia'],
       'Pożyczka':['Nie','Tak','Tak','Nie','Tak','Tak','Nie','Tak','Tak','Nie']}

In [24]:
df = pd.DataFrame(dataset)
df

,Dochód,Wiek,Wiarygodność,Pożyczka
0,niski,25,niska,Nie
1,średni,36,średnia,Tak
2,średni,36,wysoka,Tak
3,średni,25,niska,Nie
4,średni,28,średnia,Tak
5,wysoki,28,wysoka,Tak
6,niski,25,niska,Nie
7,wysoki,25,wysoka,Tak
8,średni,36,średnia,Tak
9,niski,36,średnia,Nie


### Entropia:
### $$E = - \sum_{i=1}^{N}p_{i} \cdot log_{2}p_{i}$$

gdzie $p_{i}$ jest proporcją  liczby elementów w tej grupie podziału do liczby elementów w grupie przed podziałem

In [25]:
entropy_node = 0
values = df.Pożyczka.unique()

for value in values:
    fraction = df.Pożyczka.value_counts()[value] / len(df.Pożyczka)
    entropy_node += -fraction * np.log2(fraction)
    
entropy_node    

0.9709505944546686

In [0]:
def calc_entropy(attribute):
    target_variables = df.Pożyczka.unique()
    variables = df[attribute].unique()
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] == variable][df.Pożyczka == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + epsilon)
            entropy_each_feature += -fraction * np.log2(fraction + epsilon)
        fraction2 = den / len(df)
        entropy_attribute += -fraction2 * entropy_each_feature
    entropy_attribute = abs(entropy_attribute)
    return entropy_attribute

In [0]:
def information_gain(entropy_node, entropy_attribute):
    return entropy_node - entropy_attribute

In [0]:
entropy_Dochod = calc_entropy('Dochód')
entropy_Wiek = calc_entropy('Wiek')
entropy_Wiarygodnosc = calc_entropy('Wiarygodność')

In [27]:
ig_Dochod = information_gain(entropy_node, entropy_Dochod)
ig_Wiek = information_gain(entropy_node, entropy_Wiek)
ig_Wiarygodnosc = information_gain(entropy_node, entropy_Wiarygodnosc)

print('Zysk informacyjny {}: {}'.format('Dochód', round(ig_Dochod, 4)))
print('Zysk informacyjny {}: {}'.format('Wiek', round(ig_Wiek, 4)))
print('Zysk informacyjny {}: {}'.format('Wiarygodność', round(ig_Wiarygodnosc, 4)))

Zysk informacyjny Dochód: 0.61
Zysk informacyjny Wiek: 0.3219
Zysk informacyjny Wiarygodność: 0.6464


In [0]:
def find_entropy(df):
    label = df.keys()[-1]
    entropy = 0
    values = df[label].unique()
    for value in values:
        fraction = df[label].value_counts()[value] / len(df[label])
        entropy += -fraction * np.log2(fraction)
    return entropy

def find_entropy_attribute(df, attribute):
    label = df.keys()[-1]
    target_variables = df[label].unique()
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute] == variable][df[label] == target_variable])
            den = len(df[attribute][df[attribute] == variable])
            fraction = num / (den + epsilon)
            entropy += - fraction * np.log2(fraction + epsilon)
        fraction2 = den / len(df)
        entropy2 += - fraction2 * entropy
    return abs(entropy2)
    
    
def find_winner(df):
    entropy_attr = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df) - find_entropy_attribute(df, key))
    return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def build_tree(df, tree=None):
    
    label = df.keys()[-1]
    
    node = find_winner(df)
    
    att_values = np.unique(df[node])
    
    if tree is None:
        tree = {}
        tree[node] = {}
        
    for value in att_values:
        
        subtable = get_subtable(df, node, value)
        cl_value, counts = np.unique(subtable['Pożyczka'], return_counts=True)
        
        if len(counts) == 1:
            tree[node][value] = cl_value[0]
        else:
            tree[node][value] = build_tree(subtable)
            
    return tree

In [28]:
tree = build_tree(df)
tree

{'Wiarygodność': {'niska': 'Nie',
  'wysoka': 'Tak',
  'średnia': {'Dochód': {'niski': 'Nie', 'średni': 'Tak'}}}}

In [29]:
import pprint
pprint.pprint(tree)

{'Wiarygodność': {'niska': 'Nie',
                  'wysoka': 'Tak',
                  'średnia': {'Dochód': {'niski': 'Nie', 'średni': 'Tak'}}}}


In [0]:
def predict(inst, tree):
    
    for nodes in tree.keys():
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break
    return prediction

In [31]:
inst = df.iloc[6]
inst

Dochód          niski
Wiek               25
Wiarygodność    niska
Pożyczka          Nie
Name: 6, dtype: object

In [32]:
predict(inst, tree)

'Nie'